# RAG (Retrieval-Augmented Generation) Practice (Following Tutorial)
Author: Eva Grimes 
Date: 2/28/2025

I will be following [this tutorial](https://www.youtube.com/watch?v=BNUpRW-Dk90&ab_channel=RichmondAlake) in order to get an idea of what the research project should look like

### Necessary libraries to install

In [ ]:
# !pip install datasets pandas pymongo sentence_transformers
# !pip install -U transformers
# # Install below if using GPU
# !pip install accelerate

In [ ]:
# Load Dataset
from datasets import load_dataset
import pandas as pd
# https://huggingface.co/datasets/MongoDB/embedded_movies
data = load_dataset("MongoDB/embedded_movies")
# Convert the dataset to a pandas DataFrame
data = pd.DataFrame(data['train'])

NotFoundError: Error code: 404 - {'error': {'message': 'The model `llama2` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}